In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [3]:
df= pd.read_csv("../datasets/Training.csv")
df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


### Train Test split.

In [4]:
X = df.drop('prognosis', axis=1)
y = df['prognosis']

# ecoding prognonsis
le = LabelEncoder()
le.fit(y)
Y = le.transform(y)
    
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [5]:
y_train

array([34, 34, 23, ..., 11, 15, 14])

In [6]:
input_shape = X_train.shape[1:]

In [7]:
# Create a dictionary to store models
models = {
    'SVC': SVC(kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNeighbors': KNeighborsClassifier(n_neighbors=5),
    'MultinomialNB': MultinomialNB()
}

In [8]:
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Test the model
    predictions = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model_name} Accuracy: {accuracy}")

    print("Classification Report:")
    print(classification_report(y_test, predictions))

SVC Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00        41
           3       1.00      1.00      1.00        36
           4       1.00      1.00      1.00        35
           5       1.00      1.00      1.00        36
           6       1.00      1.00      1.00        44
           7       1.00      1.00      1.00        32
           8       1.00      1.00      1.00        35
           9       1.00      1.00      1.00        30
          10       1.00      1.00      1.00        31
          11       1.00      1.00      1.00        40
          12       1.00      1.00      1.00        33
          13       1.00      1.00      1.00        45
          14       1.00      1.00      1.00        35
          15       1.00      1.00      1.00        28
          16       1.00      1.00      1

In [9]:
# selecting mode
random_forest = models['RandomForest']
random_forest.fit(X_train,y_train)
ypred = random_forest.predict(X_test)
accuracy_score(y_test,ypred)

1.0

In [10]:
import joblib
joblib.dump(model, '../models/RF_model.joblib')

['../models/RF_model.joblib']

In [13]:
# testing the model
from joblib import load
model = load("../models/RF_model.joblib")

# test 1:
print("predicted disease :",model.predict(X_test.iloc[0].values.reshape(1,-1)))
print("Actual Disease :", y_test[0])

# test 2:
print("predicted disease :",model.predict(X_test.iloc[100].values.reshape(1,-1)))
print("Actual Disease :", y_test[100])

predicted disease : [2]
Actual Disease : 2
predicted disease : [23]
Actual Disease : 23


/home/mwema/projects/smart-doctor/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
/home/mwema/projects/smart-doctor/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


# Disease Prediction System

In [14]:
sym_des = pd.read_csv("../datasets/symtoms_df.csv")
precautions = pd.read_csv("../datasets/precautions_df.csv")
workout = pd.read_csv("../datasets/workout_df.csv")
description = pd.read_csv("../datasets/description.csv")
medications = pd.read_csv('../datasets/medications.csv')
diets = pd.read_csv("../datasets/diets.csv")

In [17]:
def get_description(predicted_disease):
    return description[description['Disease'] == predicted_disease]['Description'].str.cat(sep=" ")

def get_precautions(disease):
    return precautions[precautions['Disease'] == disease][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].values.tolist()[0]

def get_medications(disease):
    return medications[medications['Disease'] == disease]['Medication'].values.tolist()

def get_diets(disease):
    return diets[diets['Disease'] == disease]['Diet'].values.tolist()

def get_workout(disease):
    return workout[workout['disease'] == disease]['workout'].values.tolist()

def helper(predicted_disease):
    desc = get_description(predicted_disease)
    pre = get_precautions(predicted_disease)
    med = get_medications(predicted_disease)
    die = get_diets(predicted_disease)
    wrkout = get_workout(predicted_disease)
    return desc, pre, med, die, wrkout

symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 
                 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 
                 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 
                 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 
                 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 
                 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 
                 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 
                 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 
                 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 
                 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 
                 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 
                 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 
                 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 
                 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 
                 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 
                 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 
                 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 
                 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 
                 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 
                 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 
                 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 
                 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 
                 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 
                 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 
                 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 
                 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 
                 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis', 14: 'Drug Reaction', 
                 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 
                 23: 'Hypertension ', 30: 'Migraine', 7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice', 
                 29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 
                 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia', 
                 13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 
                 25: 'Hypoglycemia', 31: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 
                 38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}

def get_predicted_value(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    for symptom in patient_symptoms:
        input_vector[symptoms_dict[symptom]] = 1
    predicted_disease_index = model.predict([input_vector])[0]
    return diseases_list[predicted_disease_index]


In [18]:
def diagnose_symptoms():
    # Split the user's input into a list of symptoms (assuming they are comma-separated)
    symptoms = input("Enter your symptoms: ")
    user_symptoms = [s.strip() for s in symptoms.split(',')]

    # Predict the disease based on user symptoms
    predicted_disease = get_predicted_value(user_symptoms)

    # Get details of the predicted disease
    desc, pre, med, die, wrkout = helper(predicted_disease)

    print("================= Predicted Disease ================")
    print(predicted_disease)

    print("================= Description =====================")
    print(desc)

    print("================= Precautions =====================")
    for i, p_i in enumerate(pre, start=1):
        print(f"{i}: {p_i}")

    print("================= Medications =====================")
    for j, m_i in enumerate(med, start=i):
        print(f"{j}: {m_i}")

    print("================= Workout =========================")
    for k, w_i in enumerate(wrkout, start=j):
        print(f"{k}: {w_i}")

    print("================= Diets ===========================")
    for l, d_i in enumerate(die, start=k):
        print(f"{l}: {d_i}")

# Call the function to diagnose symptoms
diagnose_symptoms()

Enter your symptoms:  yellow_crust_ooze,red_sore_around_nose,small_dents_in_nails


================= Predicted Disease ================
Impetigo
================= Description =====================
Impetigo is a highly contagious skin infection causing red sores that can break open.
================= Precautions =====================
1: soak affected area in warm water
2: use antibiotics
3: remove scabs with wet compressed cloth
4: consult doctor
================= Medications =====================
4: ['Topical antibiotics', 'Oral antibiotics', 'Antiseptics', 'Ointments', 'Warm compresses']
================= Workout =========================
4: Maintain good hygiene
5: Stay hydrated
6: Consume nutrient-rich foods
7: Limit sugary foods and beverages
8: Include foods rich in vitamin C
9: Consult a healthcare professional
10: Follow medical recommendations
11: Avoid scratching
12: Take prescribed antibiotics
13: Practice wound care
================= Diets ===========================
13: ['Impetigo Diet', 'Antibiotic treatment', 'Fruits and vegetables', 'Hydration', 'Prote

/home/mwema/projects/smart-doctor/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


In [19]:
diagnose_symptoms()

Enter your symptoms:  itching,skin_rash,nodal_skin_eruptions


================= Predicted Disease ================
Fungal infection
================= Description =====================
Fungal infection is a common skin condition caused by fungi.
================= Precautions =====================
1: bath twice
2: use detol or neem in bathing water
3: keep infected area dry
4: use clean cloths
================= Medications =====================
4: ['Antifungal Cream', 'Fluconazole', 'Terbinafine', 'Clotrimazole', 'Ketoconazole']
================= Workout =========================
4: Avoid sugary foods
5: Consume probiotics
6: Increase intake of garlic
7: Include yogurt in diet
8: Limit processed foods
9: Stay hydrated
10: Consume green tea
11: Eat foods rich in zinc
12: Include turmeric in diet
13: Eat fruits and vegetables
================= Diets ===========================
13: ['Antifungal Diet', 'Probiotics', 'Garlic', 'Coconut oil', 'Turmeric']


/home/mwema/projects/smart-doctor/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
